In [83]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train_ext = x_train.reshape(len(x_train), 28 * 28)

In [84]:
from sklearn.decomposition import PCA
pca = PCA(svd_solver='full')
pca = pca.fit(x_train_ext)

In [85]:
import numpy as np

M = 0.84
resM = -1

for a, b in enumerate(np.cumsum(pca.explained_variance_ratio_)):
    if b > M:
        resM = a + 1
        break

resM

56

In [86]:
pca_new = PCA(n_components=resM, svd_solver='full')
pca_new.fit(x_train_ext)
x_train_ext = pca_new.transform(x_train_ext)
x_train_ext.shape

(60000, 56)

In [87]:
from sklearn.model_selection import train_test_split

x_train_sp, x_test_sp, y_train_sp, y_test_sp = train_test_split(x_train_ext, y_train, test_size=0.3, random_state=95)

print(round(x_train_sp.transpose()[0].mean(),2))

2.97


In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

rfc = RandomForestClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, n_estimators=10, random_state=95)
rfc_multi = OneVsRestClassifier(rfc).fit(x_train_sp, y_train_sp)
y_predicted = rfc_multi.predict(x_test_sp)

In [93]:
from sklearn.metrics import confusion_matrix

rfc_confusion_matrix = confusion_matrix(y_test_sp, y_predicted)
print(rfc_confusion_matrix[4][4])

1615


In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

lr = LogisticRegression(solver='lbfgs', random_state=95)
lr_multi = OneVsRestClassifier(lr).fit(x_train_sp, y_train_sp)
y_predicted = lr_multi.predict(x_test_sp)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [95]:
lc_confusion_matrix = confusion_matrix(y_test_sp, y_predicted)
print(rfc_confusion_matrix[6][6])

1715


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, random_state=95)
dtc_multi = OneVsRestClassifier(dtc).fit(x_train_sp, y_train_sp)
y_predicted = dtc_multi.predict(x_test_sp)

In [ ]:
dtc_confusion_matrix = confusion_matrix(y_test_sp, y_predicted)
print(dtc_confusion_matrix[7][7])

In [97]:
import pandas as pd

digitsData = pd.read_csv("/content/drive/MyDrive/data/pred_for_task_exam.csv", delimiter = ',', index_col="FileName")
x_test = pd.DataFrame(digitsData.drop(['Label'], axis=1))
x_test = pca_new.transform(x_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but PCA was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
y_predicted = rfc_multi.predict_proba(x_test)
y_predicted.shape
round(max(y_predicted[2]),3)

In [99]:
y_predicted = lr_multi.predict_proba(x_test)
y_predicted.shape
round(y_predicted[0][7],3)

0.915

In [ ]:
y_predicted = dtc_multi.predict_proba(x_test)
y_predicted.shape
round(max(y_predicted[22]),3)

In [ ]:
digitsData